In [1]:
import os
import cv2
from glob import glob
from tqdm import tqdm
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
import json
import pandas as pd

In [ ]:
new_root = "220819"

In [ ]:
image_folders = glob("./캐플릭스데이터_부품탐색_220819_검수완료/images/*")

In [ ]:
PARTS_DF = pd.DataFrame({"part_class" : [1,2,3,4,5,6,7,8,9,10,11,12,27,13,14,15,16,17,18,19,20,21,22,23,24,25,26,28,29],
                         "color" : [[50, 50, 150], [250, 50, 250], [250, 250, 50], [150, 150, 250], 
                                  [250, 50, 150], [250, 150, 250], [150, 50, 150], [150, 150, 150], 
                                  [50, 50, 250], [250, 150, 150], [250, 250, 150], [250, 250, 250], [50, 250, 150],
                                  [250, 50, 50], [150, 150, 50], [50, 250, 250], [150, 50, 50], 
                                  [150, 250, 150], [50, 50, 50], [50, 150, 50], [50, 150, 150], 
                                  [50, 250, 50], [250, 150, 50], [150, 50, 250], [150, 250, 50], 
                                  [50, 150, 250], [150, 250, 250], [200,200,50],[255,255,255]]},
                        
                        index = ['back_door','front_bumper','front_door_left',                               
                                 'front_door_right','front_fender_left','front_fender_right',
                                 'front_fog_left','front_fog_right','front_lamp_left',
                                 'front_lamp_right','grille_up','hood',"trunk",'rear_bumper',
                                 'rear_door_left','rear_door_right','rear_fender_left',
                                 'rear_fender_right','rear_lamp_left','rear_lamp_right',
                                 'rear_stop_center','rear_stop_left','rear_stop_right',
                                 'side_mirror_left','side_mirror_right','side_step_left',
                                 'side_step_right','number_plate','minus'])

In [ ]:
part_dict = dict(dict(PARTS_DF)["part_class"])
        
drawing_order_dict = {}
for idx, x in enumerate(part_dict):
    part_data = part_dict[x]
    data = [idx, x, part_data]
    drawing_order_dict[x] = {"order":idx,
                             "part_id":part_data}
    
part_color = dict(dict(PARTS_DF)["color"])

In [ ]:
def sort_to_draw_order(annotation_info):
        
    draw_order = []
    for ai in annotation_info:
        
        category_name = ai["label"].lower()
        # print(category_name)
        
        if category_name == "backdoor":
            category_name = "back_door"

        draw_data = drawing_order_dict[category_name]

        draw_order_num = draw_data["order"]
        part_color_id = draw_data["part_id"]

        # print(draw_order_num, category_name, part_color_id)
        draw_order.append([draw_order_num, category_name, part_color_id, ai])

    draw_order = sorted(draw_order, key=lambda x: x[0])

    return draw_order

In [ ]:
def convert_to_png(img_path, annot_info):
    
    image_data = cv2.imread(img_path)
    image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
    
    h,w,_ = image_data.shape
    
    white_paper = np.full((h,w,3),255).astype(np.uint8)
    
    draw_info = sort_to_draw_order(annot_info)

    for di in draw_info:
        
        _, category_name, part_id, segmentation_data = di

        color = part_color[category_name]

        # polygon = np.array(segmentation_data["segmentation"]).reshape((1,-1,2)).astype(np.int32)
        
        polygon = np.array(segmentation_data["points"]).astype(np.int32)

        cv2.fillPoly(white_paper, pts=[polygon], color=color)
    
    return white_paper

In [ ]:
def read_json(config_path):
    """
    json 파일을 읽어 dict 형태로 반환해주는 함수
    """
    with open(config_path, encoding="utf-8-sig") as json_file:
        json_data = json.load(json_file)
    return json_data

In [ ]:
def label_drawer(img_path, annot_path):
    
    json_data = read_json(annot_path)
    
    labeling_data = json_data["shapes"]
    
    label_image = convert_to_png(img_path, labeling_data)
    
    return label_image

In [ ]:
for imf in image_folders:
    
    images = glob(os.path.join(imf, ("*.jpg")))
    
    refined_data = []
    
    for i in tqdm(images):
    
        a = i.replace("images","annotations").replace("jpg","json")
    
        if os.path.exists(a):
            
            refined_data.append((i,a))
    if len(refined_data) > 5:
    
        train, test = train_test_split(refined_data, test_size=5, random_state=12345 )
        
    else:
        train, test = train_test_split(refined_data, test_size=2, random_state=12345)
    
    for h in ["train","eval"]:
        for b in ["images","annotations"]:
            new_path = os.path.join(new_root,h,b)
            os.makedirs(new_path,exist_ok=True)
            
    for tr in train:
        
        imgs, annots = tr
        
        label_image = label_drawer(imgs,annots)
        
        im_basename = os.path.basename(imgs)
        annots_basename = os.path.basename(annots).replace("json","png")
        
        new_im_path = os.path.join(new_root, "train", "images", im_basename)
        new_annot_path = os.path.join(new_root,"train","annotations",annots_basename)
        
        shutil.copy2(imgs, new_im_path)
        # shutil.copy2(annots, new_annot_path)
        cv2.imwrite(new_annot_path, cv2.cvtColor(label_image, cv2.COLOR_RGB2BGR) )
        
    for ev in test:
        
        imgs, annots = ev
        
        label_image = label_drawer(imgs,annots)
        
        im_basename = os.path.basename(imgs)
        annots_basename = os.path.basename(annots).replace("json","png")
        
        new_im_path = os.path.join(new_root, "eval", "images", im_basename)
        new_annot_path = os.path.join(new_root,"eval","annotations",annots_basename)
        
        shutil.copy2(imgs, new_im_path) 
        # shutil.copy2(annots, new_annot_path)
        cv2.imwrite(new_annot_path, cv2.cvtColor(label_image, cv2.COLOR_RGB2BGR) )
        

In [ ]:
annots

In [3]:
len(glob('../twincar-part-kaflix/eval/images/*'))

267

In [4]:
len(glob('../twincar-part-kaflix/train/images/*'))

3740